In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
from scipy.stats import multivariate_normal
pd.set_option('display.max_columns', None)  


In [ ]:
mypath = 'Datasets/Gaze10minData/'
mydtaframes = {}

for index,files in enumerate(os.listdir(mypath)):
    mydtaframes["df_{}".format(index)] = pd.read_excel('{}/{}'.format(mypath, files),
                                                         sheet_name= 'Sheet1')

In [ ]:
mydtaframes['df_1'].head()

In [102]:
studiodf = {}

for index,frames in enumerate(mydtaframes):
    studiodf["{}".format(index)] = mydtaframes[frames]
    #studiodf["{}".format(index)] = mydtaframes[frames].loc[(mydtaframes[frames]['StudioEventTypeDiff'] == "Agree_1_Scanning") | (mydtaframes[frames]['StudioEventTypeDiff'] == "Agree_2_Skimming") | (mydtaframes[frames]['StudioEventTypeDiff'] == "Agree_3_Reading"),['LocalTimeStamp','FixationPointX (MCSpx)','FixationPointY (MCSpx)','StudioEventTypeDiff']]
# studiodf = df1.loc[(df1['StudioEventTypeDiff'] == "Agree_1_Scanning") | (df1['StudioEventTypeDiff'] == "Agree_2_Skimming") | (df1['StudioEventTypeDiff'] == "Agree_3_Reading"),['LocalTimeStamp','FixationPointX (MCSpx)','FixationPointY (MCSpx)','StudioEventTypeDiff']]
# studiodf.head()

In [103]:
len(studiodf)

13

In [104]:
for frames in studiodf:
    studiodf[frames].fillna(method='ffill', inplace= True)

In [106]:

allData = pd.concat([studiodf['0'],studiodf['1'],studiodf['2'],studiodf['3'],studiodf['4'],studiodf['5'],
                    studiodf['6'],studiodf['7'],studiodf['8'],studiodf['9'],studiodf['10'],studiodf['11'],
                    studiodf['12'],], sort = False)
allData['GazePointX (MCSpx)'].fillna(method='bfill', inplace= True)
allData['GazePointY (MCSpx)'].fillna(method='bfill', inplace= True)

In [107]:
allData['GazePointX (MCSpx)'] = allData['GazePointX (MCSpx)'].diff()
allData['GazePointY (MCSpx)'] = allData['GazePointY (MCSpx)'].diff()
allData['GazePointX (MCSpx)'].fillna(method='bfill', inplace= True)
allData['GazePointY (MCSpx)'].fillna(method='bfill', inplace= True)


In [108]:
allData['GazePointX (MCSpx)'].isna().sum().sum()

0

In [109]:
allData['StudioEventTypeDiff'].unique()

array(['Agree_0_unstated', 'Disagree_5_Unknown_0_unstated',
       'Disagree_5_Unknown_3_Reading', 'Disagree_1_Scanning_3_Reading',
       'Disagree_1_Scanning_5_Unknown', 'Agree_5_Unknown',
       'Disagree_5_Unknown_1_Scanning', 'Agree_1_Scanning',
       'Agree_3_Reading', 'Disagree_1_Scanning_2_Skimming',
       'Disagree_5_Unknown_2_Skimming', 'Agree_2_Skimming',
       'Disagree_1_Scanning_4_MediaView',
       'Disagree_2_Skimming_1_Scanning', 'Disagree_2_Skimming_5_Unknown',
       'Disagree_2_Skimming_3_Reading', 'Disagree_5_Unknown_4_MediaView',
       'Agree_4_MediaView', 'Disagree_4_MediaView_1_Scanning',
       'Disagree_0_unstated_3_Reading', 'Disagree_3_Reading_1_Scanning',
       'Disagree_3_Reading_2_Skimming', 'Disagree_2_Skimming_4_MediaView',
       'Disagree_3_Reading_4_MediaView',
       'Disagree_4_MediaView_2_Skimming',
       'Disagree_4_MediaView_5_Unknown', 'Disagree_3_Reading_5_Unknown',
       'Disagree_1_Scanning_0_unstated', 'Disagree_0_unstated_5_Unknown'

In [110]:
import matplotlib.pyplot as plt

In [111]:
from IPython.display import display

In [112]:
readingData = allData.loc[allData['StudioEventTypeDiff'] == "Agree_3_Reading"]
skimmingData = allData.loc[allData['StudioEventTypeDiff'] == "Agree_2_Skimming"]
scanningData = allData.loc[allData['StudioEventTypeDiff'] == "Agree_1_Scanning"]
unknownData = allData.loc[allData['StudioEventTypeDiff'] == "Agree_5_Unknown"]
mediaViewData = allData.loc[allData['StudioEventTypeDiff'] == "Agree_4_MediaView"]

In [113]:
readingData['GazePointY (MCSpx)'].isna().sum().sum()

0

In [114]:
readingData.head()

,ParticipantName,RecordingDuration,RecordingTimestamp,LocalTimeStamp,EyeTrackerTimestamp,MouseEventIndex,MouseEventX (MCSpx),MouseEventY (MCSpx),KeyPressEventIndex,StudioEventIndex,StudioEventIndex.1,StudioEvent,StudioEvent_B,FixationIndex,SaccadeIndex,GazeEventType,GazeEventType_B,GazeEventDuration,FixationPointX (MCSpx),FixationPointY (MCSpx),AbsoluteSaccadicDirection,RelativeSaccadicDirection,GazePointIndex,GazePointX (MCSpx),GazePointY (MCSpx),PupilLeft,PupilRight,ValidityLeft,ValidityRight,StudioEventTypeDiff,GazeEventTypeDiff,StudioEventIndex_B,StudioEventIndex_B.1,Unnamed: 11
4691,Proband_30_D1,3520531,77615,14:47:02.446,1.457961e+15,14.0,364.0,13.0,52.0,8.0,8.0,3_Reading,3_Reading,206.0,410.0,Fixation,Fixation,250,162.0,451.0,"348,49","172,21",4640.0,0.0,0.0,"2,25","2,43",0.0,0.0,Agree_3_Reading,Agree_Fixation,NaN,NaN,NaN
4692,Proband_30_D1,3520531,77629,14:47:02.460,1.457961e+15,14.0,364.0,13.0,52.0,8.0,8.0,3_Reading,3_Reading,206.0,410.0,Fixation,Fixation,250,162.0,451.0,"348,49","172,21",4641.0,2.0,-1.0,"2,29","2,43",0.0,0.0,Agree_3_Reading,Agree_Fixation,NaN,NaN,NaN
4693,Proband_30_D1,3520531,77645,14:47:02.476,1.457961e+15,14.0,364.0,13.0,52.0,8.0,8.0,3_Reading,3_Reading,206.0,410.0,Fixation,Fixation,250,162.0,451.0,"348,49","172,21",4642.0,-6.0,21.0,"2,26","2,41",0.0,0.0,Agree_3_Reading,Agree_Fixation,NaN,NaN,NaN
4694,Proband_30_D1,3520531,77662,14:47:02.493,1.457961e+15,14.0,364.0,13.0,52.0,8.0,8.0,3_Reading,3_Reading,206.0,410.0,Fixation,Fixation,250,162.0,451.0,"348,49","172,21",4643.0,11.0,-20.0,"2,24","2,39",0.0,0.0,Agree_3_Reading,Agree_Fixation,NaN,NaN,NaN
4695,Proband_30_D1,3520531,77679,14:47:02.510,1.457961e+15,14.0,364.0,13.0,52.0,8.0,8.0,3_Reading,3_Reading,206.0,410.0,Fixation,Fixation,250,162.0,451.0,"348,49","172,21",4644.0,0.0,6.0,"2,28","2,41",0.0,0.0,Agree_3_Reading,Agree_Fixation,NaN,NaN,NaN


In [115]:
readCovMat = np.cov((readingData['GazePointX (MCSpx)'],readingData['GazePointY (MCSpx)']))
skimmingCovMat = np.cov((skimmingData['GazePointX (MCSpx)'],skimmingData['GazePointY (MCSpx)']))
scanningCovMat = np.cov((scanningData['GazePointX (MCSpx)'],scanningData['GazePointY (MCSpx)']))
unknownCovMat = np.cov((unknownData['GazePointX (MCSpx)'],unknownData['GazePointY (MCSpx)']))
mediaViewCovMat = np.cov((mediaViewData['GazePointX (MCSpx)'],mediaViewData['GazePointY (MCSpx)']))

display(readCovMat)
display(skimmingCovMat)
display(scanningCovMat)
display(unknownCovMat)
display(mediaViewCovMat)

array([[1179.63428727,  171.67930601],
       [ 171.67930601,  836.91103656]])

array([[1805.68290536,  216.39954858],
       [ 216.39954858, 4072.86681401]])

array([[1852.15462508,  330.37926778],
       [ 330.37926778, 1716.64905786]])

array([[1786.47885221,  598.16561454],
       [ 598.16561454, 2461.48454459]])

array([[1922.47066957, -383.06551818],
       [-383.06551818, 1367.8950435 ]])

In [116]:
readingMeanx = readingData['GazePointX (MCSpx)'].mean()
readingMeany = readingData['GazePointY (MCSpx)'].mean()
readingMeanVector = [readingMeanx,readingMeany]

skimmingMeanx = skimmingData['GazePointX (MCSpx)'].mean()
skimmingMeany = skimmingData['GazePointY (MCSpx)'].mean()
skimmingMeanVector = [skimmingMeanx,skimmingMeany]

scanningMeanx = scanningData['GazePointX (MCSpx)'].mean()
scanningMeany = scanningData['GazePointY (MCSpx)'].mean()
scanningMeanVector = [scanningMeanx,scanningMeany]

unknownMeanx = unknownData['GazePointX (MCSpx)'].mean()
unknownMeany = unknownData['GazePointY (MCSpx)'].mean()
unknownMeanVector = [unknownMeanx,unknownMeany]

mediaViewMeanx = mediaViewData['GazePointX (MCSpx)'].mean()
mediaViewMeany = mediaViewData['GazePointY (MCSpx)'].mean()
mediaViewMeanVector = [mediaViewMeanx,mediaViewMeany]

display(readingMeanVector)
display(skimmingMeanVector)
display(scanningMeanVector)
display(unknownMeanVector)
display(mediaViewMeanVector)


[-0.018223117030612773, 0.04327775196599728]

[0.3212, -0.6845777777777777]

[-0.14131410896028737, -0.08072241069646777]

[-0.04316383904262544, 0.06364754324031643]

[-1.0968448729184925, 0.2287467134092901]

In [117]:
readingPassData = readingData[['GazePointX (MCSpx)','GazePointY (MCSpx)']]
skimmingPassData = skimmingData[['GazePointX (MCSpx)','GazePointY (MCSpx)']]
scanningPassData = scanningData[['GazePointX (MCSpx)','GazePointY (MCSpx)']]
unknownPassData = unknownData[['GazePointX (MCSpx)','GazePointY (MCSpx)']]
mediaViewPassData = mediaViewData[['GazePointX (MCSpx)','GazePointY (MCSpx)']]

In [118]:
readPdf = multivariate_normal.pdf(readingPassData, mean=readingMeanVector, cov=readCovMat)
skimmingPdf = multivariate_normal.pdf(skimmingPassData, mean=skimmingMeanVector, cov=skimmingCovMat)
scanningPdf = multivariate_normal.pdf(scanningPassData, mean=scanningMeanVector, cov=scanningCovMat)
unknownPdf = multivariate_normal.pdf(unknownPassData, mean=unknownMeanVector, cov=unknownCovMat)
mediaViewPdf = multivariate_normal.pdf(mediaViewPassData, mean=mediaViewMeanVector, cov=mediaViewCovMat)

In [ ]:
plt.plot(readingPassData,readPdf)

In [ ]:
plt.plot(skimmingPassData,skimmingPdf)

In [ ]:
plt.plot(scanningPassData,scanningPdf)

In [ ]:
from hidden_markov import hmm

In [ ]:
states = ('Scanning','Skimming','Reading')
observations = ('FixationPointX (MCSpx)','FixationPointY (MCSpx)','GazeEventType')
start_probability = np.matrix('0 0 1')
transition_probability = np.matrix('0.2 0.3 0.5; 0.3 0.6 0.1; 0.5 0.2 0.3')
emission_pprobability = np.matrix('0.1 0.4 0.5; 0.6 0.1 0.3; 0.1 0.7 0.2')
test = hmm(states, observations, start_probability, transition_probability, emission_pprobability)

obs = ('FixationPointX (MCSpx)','FixationPointX (MCSpx)','GazeEventType')

In [ ]:
test.viterbi(obs)